In this codebook we will focus on creating new variables using pandas methods and functions.This codebook demonstrates the following

- Converting string to numeric using pd.to_numeric
- Creating dummy variables using pd.get_dummies
- Creating numeric variables from string using str methods
- Creating categorical variables from numeric variables using pd.cut 

In [1]:
%pwd

'C:\\Users\\Shravan\\Downloads\\Data management and manipulation'

In [3]:
import pandas as pd

In [7]:
tp = pd.read_csv('zomato.csv', chunksize =1000,sep=',')


In [8]:
new = pd.concat(tp, ignore_index=True)


In [9]:
df=new.copy()

Getting insights of the data 

In [10]:
df.shape

(51717, 17)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
url                            51717 non-null object
address                        51717 non-null object
name                           51717 non-null object
online_order                   51717 non-null object
book_table                     51717 non-null object
rate                           43942 non-null object
votes                          51717 non-null int64
phone                          50509 non-null object
location                       51696 non-null object
rest_type                      51490 non-null object
dish_liked                     23639 non-null object
cuisines                       51672 non-null object
approx_cost(for two people)    51371 non-null object
reviews_list                   51717 non-null object
menu_item                      51717 non-null object
listed_in(type)                51717 non-null object
listed_in(city)                51717 non-nul

In [ ]:
df.columns

In [12]:
# deleting unnecessary data
del df['url']
del df['address']
del df['phone']
del df['reviews_list']
del df['menu_item']

In [ ]:
# here is the new set of columns
df.columns

Its time to rename the columns

In [46]:
# renaming features 
df = df.rename(columns = {"approx_cost(for two people)" : "avg_cost", "listed_in(type)" : "meal_type", 
                   "listed_in(city)" : "city"})

In [14]:
# here is the new set of features
df.columns

Index(['name', 'online_order', 'book_table', 'rate', 'votes', 'location',
       'rest_type', 'dish_liked', 'cuisines', 'avg_cost', 'meal_type', 'city'],
      dtype='object')

In [15]:
# Missing value 
df.isnull().sum()


name               0
online_order       0
book_table         0
rate             800
votes              0
location           1
rest_type         27
dish_liked      2625
cuisines           4
avg_cost          46
meal_type          0
city               0
dtype: int64

Dish_liked has more than 50% missing values.Lets drop it 

In [47]:
del df['dish_liked']

In [ ]:
df.columns

### The rate feature 

In [ ]:
df.head()

In [ ]:
type('rate')

There are two ways of extracting the rate out of the string 

- Using regex we can just take the first 3 digits 
- ALternatively we can use string manipulation using str.split

In [ ]:
df['rate1'] = df['rate'].str.extract('(^\d\.\d)') # Using regex


In [ ]:
df['rate1'].head()

In [ ]:
type('rate1')

It is still a string.It needs to be converted into a number for any useful analysis.This can be done in two ways 

- astype
- pd.to_numeric

to_numeric however has a added advantage.In case the rate variable has unexpected values such as # $ etc. to_numeric with the coerce option will automatically convert it into missing values

In [ ]:
df['rate1']=pd.to_numeric(df['rate1'],errors='coerce')

In [ ]:
df['rate1'].isnull().sum() # HOw much was it earlier? The missing value has increased from 7K to 10K

Now the rate variable is ready for a missing value treatment. 

### Creating useful variables

#### Dummies from binary variables

In [ ]:
df.columns

In [ ]:
df['online_order'].value_counts()

In [ ]:
df['online_order']= pd.get_dummies(df['online_order'],drop_first=True)

In [ ]:
df['rest_type'].isnull().sum()

In [ ]:
df['rest_type1']=df['rest_type'].fillna('Casual')

In [ ]:
df.isnull().sum()

#### New variable by rectegorizing rest_type

In [ ]:
df[df['rest_type1'].str.contains('Quick|Takeaway|Casual|Cafe|Sweet|Dessert|Bakery')]

In [ ]:
df.loc[df.rest_type1.str.contains('Quick|Takeaway|Casual|Mess|Bhojanalya|Truck|Dhaba|Kiosk|Court'),'rest_type1']='Casual'

In [ ]:
df.loc[df.rest_type1.str.contains('Cafe|Sweet|Dessert|Bakery|Confectionery|Beverage'),'rest_type1']='Snacks'

In [ ]:
df.loc[df.rest_type1.str.contains('Bar|Pub|Microbrewery|Lounge'),'rest_type1']='Bar'

In [ ]:
df.loc[df.rest_type1.str.contains('Fine|Club'),'rest_type1']='Fine Dining'

In [ ]:
df.loc[df.rest_type1.str.contains('Delivery'),'rest_type1']='Delivery'

In [13]:
df['rest_type1'].value_counts()

KeyError: 'rest_type1'

Now we can further convert these variables  into dummies .Since there are 6 categories how many dummies will we have ? 

#### Creating numeric from categorical variables by counts

Can we find the no of cuisines ? 

In [ ]:
df['cuisines'].str.split(",",expand = True) # Using string manipulation # tahe expand option takes the split values and assigns them into individual columns

In [ ]:
df['no_cuisines']=df['cuisines'].str.split(",",expand = True).count(axis=1) # When we add count() it gives a count of cells across the row

In [ ]:
#If I do want to retain seperate columns than I have to create them

df[['cuisine1','cuisine2','cuisine3','cuisine4','cuisine5','cuisine5','cuisine7','cuisine8']]=df['cuisines'].str.split(",",expand = True)

In [ ]:
df.head()

In [ ]:
# new data frame with split value columns 
df[['cuisines','no_cuisines']].head()

In [ ]:
df.columns

#### Creating categorical variables from numerical variables 

In [ ]:
df1=df.copy()

Categorical variables can be created by using conditional if statements.HOWever a conditional loop applied on a large dataset can be memory intensive.Pandas offers a better alternative through pd.cut function

In [53]:
df['avg_cost1']=pd.to_numeric(df['avg_cost'],errors='coerce')

In [59]:
df['avg_cost1']=df['avg_cost1'].fillna(400)

In [60]:
df['avg_cost1'].describe(percentiles=[.1, .2,.3,.4,.5,.6,.7,.8,.9,1])

count    51717.000000
mean       414.294623
std        180.524316
min         40.000000
10%        200.000000
20%        250.000000
30%        300.000000
40%        400.000000
50%        400.000000
60%        400.000000
70%        500.000000
80%        600.000000
90%        700.000000
100%       950.000000
max        950.000000
Name: avg_cost1, dtype: float64

In [63]:
df['price_cat'] = pd.cut(x=df['avg_cost1'], bins=[0,50,100,200, 400, 600, 800,1000],labels=['0-50','50-100','100-200','200-400','400-600','600-800','800-1000'])


In [57]:
df.shape

(51717, 13)

In [66]:
df['price_cat'].value_counts(dropna=False)

200-400     26141
400-600     10874
100-200      6984
600-800      5767
50-100       1007
800-1000      928
0-50           16
Name: price_cat, dtype: int64

References :

http://buildingskills.itmaybeahack.com/book/programming-2.6/html/p13_modules/p13_c05_regex.html
    https://chrisalbon.com/python/data_wrangling/pandas_regex_to_create_columns/